# Test RedisAI with Pytorch

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

## Create the network

Take the network from the pytorch MNIST examples 
(https://github.com/pytorch/examples/blob/master/mnist/main.py)

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        output = F.log_softmax(x, dim=1)
        return output

## Create the dataset and the data loader

In [3]:
transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
        ])

data = datasets.MNIST('./data', train=True, download=False, transform=transform)

In [4]:
train_loader = torch.utils.data.DataLoader(data, batch_size=1)

In [5]:
it = iter(train_loader)

### Load a single tensor to forward

In [6]:
img, target = it.next()

## Create the network and do a forward and backward pass to get the gradients

In [7]:
model = Net()
optimizer = optim.Adam(model.parameters(), lr=0.01)
model.train()

Net(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1))
  (dropout1): Dropout(p=0.25, inplace=False)
  (dropout2): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=9216, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=10, bias=True)
)

In [8]:
# Zero the optimizer before the forward pass
optimizer.zero_grad()

# forward pass
out = model(img)

loss = F.nll_loss(out, target)
loss.backward()

In [9]:
c1_grad = model.conv1.weight.grad
c1_bias = model.conv1.bias.grad

In [13]:
model.conv2.bias

Parameter containing:
tensor([ 0.0227, -0.0067, -0.0178, -0.0109, -0.0477, -0.0449,  0.0397, -0.0082,
         0.0264,  0.0336, -0.0039, -0.0088, -0.0036, -0.0117, -0.0381, -0.0327,
         0.0582, -0.0235,  0.0553,  0.0312, -0.0406,  0.0068, -0.0255, -0.0035,
        -0.0088, -0.0571, -0.0233,  0.0372,  0.0306, -0.0292, -0.0360, -0.0016,
        -0.0143, -0.0280,  0.0211, -0.0314,  0.0349, -0.0057,  0.0332, -0.0252,
        -0.0051, -0.0295, -0.0579,  0.0104, -0.0027, -0.0251, -0.0517,  0.0363,
        -0.0509, -0.0433, -0.0313,  0.0249, -0.0252, -0.0375, -0.0479, -0.0196,
         0.0496,  0.0129,  0.0549,  0.0393, -0.0151,  0.0468, -0.0266,  0.0164],
       requires_grad=True)

## Start with the REDIS AI part

In [14]:
import redisai as rai

In [15]:
con = rai.Client(host='192.168.99.102', port=6379)

In [17]:
con.tensorset('grad-conv1', c1_grad.numpy(), dtype='float32')
con.tensorset('bias-conv1', c1_bias.numpy(), dtype='float32')

'OK'

In [18]:
con.set('example', 'hola')
con.set('exaaaaa', 'hola2')

True